In [1]:
%%time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout, BatchNormalization, MaxPool2D
from IPython.display import Image, display
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.datasets import make_blobs
from sklearn.model_selection import LeaveOneOut
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import *

Wall time: 1.69 s


In [2]:
data_folder = os.listdir(r'D:\ILBS_14_8_21_Sanjeev\ILBS_Cropped_Volumes')

In [3]:
%%time
all_image = []
path = 'D:\\ILBS_14_8_21_Sanjeev\\ILBS_Cropped_Volumes'
for folder in data_folder:
    images = []
    IMG_HEIGHT = IMG_WIDTH =196
    for row in os.listdir(path+'\\'+folder):
        fpath = path+'\\'+folder+'\\'+row
        #print(fpath)
        image= cv2.imread( fpath, cv2.IMREAD_GRAYSCALE)
        image=cv2.resize(image, (IMG_HEIGHT, IMG_WIDTH),interpolation = cv2.INTER_AREA)
        max_val = np.max(image)
        image=np.array(image)
        image = image.astype('float32')
        images.append(image)
    mean_vol = np.mean(images)
    dev_vol = np.std(images)
    images = (images-mean_vol)/dev_vol
    all_image.append(np.array(images).reshape([IMG_HEIGHT,IMG_WIDTH,150]))

Wall time: 57min 47s


In [4]:
%%time
merged_data = np.array(all_image)

Wall time: 606 ms


In [5]:
merged_data[0].shape

(196, 196, 150)

In [6]:
X = merged_data

In [7]:
X.shape

(99, 196, 196, 150)

In [8]:
df_f = pd.read_excel('D:/ILBS_14_8_21_Sanjeev/Work_to_be_Completed_ILBS/HVPG value with CT clustered.xlsx',engine='openpyxl')

In [9]:
df_f.head(n=10)

,ID,Date,HVPG
0,9109.0,2019-01-30,4.0
1,19699.0,2019-02-13,2.0
2,31322.0,2019-02-15,3.0
3,45341.0,2019-02-13,3.0
4,49565.0,2019-02-06,1.0
5,95470.0,2019-02-12,3.0
6,99036.0,2019-02-08,3.0
7,110991.0,2019-01-31,2.0
8,115428.0,2019-02-05,3.0
9,119394.0,2019-01-23,3.0


In [25]:
y = df_f['HVPG'][0:99]

In [11]:
df_f['HVPG'][99]

nan

In [26]:
y= np.array(y)

In [27]:
y.shape

(99,)

In [15]:
%%time
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', 
                 input_shape=(196, 196,150)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(5, activation='softmax'))

Wall time: 2.78 s


In [19]:
from sklearn.model_selection import LeaveOneOut
cv = LeaveOneOut()
for train_ix, test_ix in cv.split(X):
    print(train_ix)

[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48
 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72
 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96
 97 98]
[ 0  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48
 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72
 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96
 97 98]
[ 0  1  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48
 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72
 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96
 97 98]
[ 0  1  2  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33

In [28]:
from sklearn.model_selection import LeaveOneOut
cv = LeaveOneOut()
test_score_l = []
hist = []
for train_ix, test_ix in cv.split(X):
    #print(train_ix)
    X_train, X_test = X[train_ix, :], X[test_ix, :]
    y_train, y_test = y[train_ix], y[test_ix]
    #print(y_train)
    y = np.array(y_train)
    y_train = keras.utils.to_categorical(y)
    y_t = np.array(y_test)
    y_test = keras.utils.to_categorical(y_t)
    model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])
    history = model.fit(X_train, y_train, epochs=15, verbose=1,shuffle=True)
    hist.append(history)
    test_score = model.evaluate(X_test, y_test, verbose = 1)
    test_score_l.append(test_score)
    

Train on 98 samples
Epoch 1/15
98/98 [==============================] - 168s 2s/sample - loss: 1.6818 - accuracy: 0.3265
Epoch 2/15
32/98 [========>.....................] - ETA: 3s

ResourceExhaustedError: 2 root error(s) found.
  (0) Resource exhausted:  OOM when allocating tensor with shape[32,196,196,150] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu
	 [[{{node GatherV2}}]]
	 [[IteratorGetNext]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[Shape/_10]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

  (1) Resource exhausted:  OOM when allocating tensor with shape[32,196,196,150] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu
	 [[{{node GatherV2}}]]
	 [[IteratorGetNext]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

0 successful operations.
0 derived errors ignored. [Op:__infe

In [24]:
X.shape

(99, 196, 196, 150)

In [25]:
y.shape

(99,)